In [1]:
import torch
import cv2  
from torchvision import models, transforms
import numpy as np

In [2]:
model = models.segmentation.deeplabv3_resnet101(pretrained=True).eval()

C:\Users\fluri\AppData\Roaming\Python\Python311\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\fluri\AppData\Roaming\Python\Python311\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DeepLabV3_ResNet101_Weights.COCO_WITH_VOC_LABELS_V1`. You can also use `weights=DeepLabV3_ResNet101_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [3]:
def segment(frame, orig_dims):
    preprocess = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((256, 256)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

    input_tensor = preprocess(frame)
    input_batch = input_tensor.unsqueeze(0)

    if torch.cuda.is_available():
        input_batch = input_batch.to('cuda')
        model.to('cuda')
    
    with torch.no_grad():
        output = model(input_batch)['out'][0]
    output_predictions = output.argmax(0)

    mask = output_predictions.byte().cpu().numpy()
    mask = cv2.resize(mask, (orig_dims[1], orig_dims[0]))
    return mask

In [4]:
cap = cv2.VideoCapture(r'C:\Users\fluri\Pictures\Camera Roll\1234.mp4')
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
orig_dims = (frame_height, frame_width)


In [5]:
out = cv2.VideoWriter('output1.avi', cv2.VideoWriter_fourcc('M', 'J', 'P', 'G'), 10, (frame_width, frame_height))

In [6]:
while(cap.isOpened()):
    ret, frame = cap.read()
    if not ret:
        break
    mask = segment(frame, orig_dims)
    frame[mask == 0] = [0, 0, 0]
    out.write(frame)

In [7]:
cap.release()
out.release()
cv2.destroyAllWindows()